In [1]:
# Import required tools
import pandas as pd
pd.set_option('display.max_columns', None)
from pandas import datetime
import seaborn as sns
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from ipywidgets import interact
import matplotlib.pyplot as plt
%matplotlib inline
import pyodbc
import adodbapi
%load_ext sql
import numpy as np
import plotly
import plotly.graph_objs as go
import plotly.offline as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import ipywidgets as widgets
from ipywidgets import interactive, HBox, VBox, HTML, ToggleButtons, interact, interact_manual

In [75]:
df = pd.read_csv('E:/Downloads/Credit_risk.csv')
df.head()

,LEI_code,NSA,Periods,Item,Portfolio,Country,Country_rank,Exposure,Status,Perf_Status,AMOUNT
0,PQOH26KWDF7CG10L6792,AT,201412,150501,1,0,0,0,0,0,71177.046165
1,PQOH26KWDF7CG10L6792,AT,201412,150502,1,0,0,103,0,0,34204.300706
2,PQOH26KWDF7CG10L6792,AT,201412,150502,1,0,0,104,0,0,4944.808301
3,PQOH26KWDF7CG10L6792,AT,201412,150502,1,0,0,105,0,0,3364.781241
4,PQOH26KWDF7CG10L6792,AT,201412,150502,1,0,0,106,0,0,550.031529


In [8]:
df.dtypes

LEI_code         object
NSA              object
Periods           int64
Item              int64
Portfolio         int64
Country           int64
Country_rank      int64
Exposure          int64
Status            int64
Perf_Status       int64
AMOUNT          float64
dtype: object

In [9]:
df.shape

(251666, 11)

In [20]:
Total_Bucket = df.groupby(['Periods','NSA'])['AMOUNT'].agg(['mean', 'sum'])

Total_Bucket.head(10)

mean           sum
Periods NSA                           
201412  AT    868.563426  6.181566e+06
        BE    873.300176  7.733073e+06
        CY    385.224793  1.042418e+06
        DE   1324.412992  3.701999e+07
        DK   2180.487687  9.083912e+06
        ES   5357.874904  3.903748e+07
        FI    844.143286  1.494978e+06
        FR   4199.953606  6.268011e+07
        HU    586.248864  5.287965e+05
        IE    999.029037  5.307841e+06

In [82]:
df1 = pd.read_csv('E:/Downloads/fsi.csv')

df1.head()

,Date,OFR_FSI,Credit,Equity_valuation,Safe_assets,Funding,Volatility,United_States,Other_advanced_economies,Emerging_markets
0,2000-01-03,2.140,0.540,-0.051,0.670,0.472,0.509,1.769,0.521,-0.150
1,2000-01-04,2.421,0.604,0.079,0.627,0.550,0.561,2.084,0.474,-0.137
2,2000-01-05,2.297,0.575,0.080,0.653,0.501,0.488,2.023,0.406,-0.132
3,2000-01-06,2.292,0.546,0.082,0.581,0.566,0.517,2.064,0.355,-0.127
4,2000-01-07,2.005,0.662,-0.011,0.519,0.565,0.270,1.769,0.367,-0.131


In [83]:
df1.dtypes

Date                         object
OFR_FSI                     float64
Credit                      float64
Equity_valuation            float64
Safe_assets                 float64
Funding                     float64
Volatility                  float64
United_States               float64
Other_advanced_economies    float64
Emerging_markets            float64
dtype: object

In [84]:
df1.isnull().count()

Date                        5125
OFR_FSI                     5125
Credit                      5125
Equity_valuation            5125
Safe_assets                 5125
Funding                     5125
Volatility                  5125
United_States               5125
Other_advanced_economies    5125
Emerging_markets            5125
dtype: int64

In [85]:
df1.reset_index(inplace=True)

In [113]:
f = go.FigureWidget(
    data = [go.Scattergl(x = df1['Date'],
                         y = df1['OFR_FSI'],
                         marker=dict(
                             color='grey', size = 12,
                             line=dict(color = 'rgb(231, 99, 250)',
                                       width = 4)),
                         name='FSI',
                         yaxis='y1'),                        
            go.Scattergl(x = df1['Date'],
                         y = df1['Funding'],
                         fill='tonexty',
                         marker=dict(
                             color='yellow', size = 12,
                             line=dict(color = 'rgb(231, 99, 250)',
                                       width = 4)),
                   name='Funding',
                   yaxis='y1'),
            go.Scattergl(x = df1['Date'],
                         y = df1['Safe_assets'],
                         marker=dict(
                             color='green', size = 12,
                             line=dict(color = 'rgb(231, 99, 250)',
                                       width = 4)),
                         name='Safe Assets',
                         yaxis='y1'),
            go.Scatter(x = df1['Date'],
                       y = df1['Volatility'],
                       fill='tonexty',
                       marker=dict(
                           color='orange', size = 12,
                           line=dict(color = 'rgb(231, 99, 250)',
                                     width = 4)),
                       name='Volatility',
                       yaxis='y1'),
            go.Scattergl(x = df1['Date'],
                         y = df1['Credit'],
                         fill='tonexty',
                         marker=dict(
                             color='#1e4aa6', size = 12,
                             line=dict(color = 'rgb(231, 99, 250)',
                                       width = 4)),
                         name='Credit',
                         yaxis='y1',
                         mode = 'lines'),],
    layout = go.Layout(title = 'MUFG Programing Sample',
                       xaxis={"title":"Time Range",
                              'rangeselector': {'buttons': list([
                                  {'count': 1, 'label': '1 week', 'step': 'week', 'stepmode': 'backward'},
                                  {'count': 1, 'label': '1M', 'step': 'month', 'stepmode': 'backward'},
                                  {'count': 6, 'label': '6M', 'step': 'month', 'stepmode': 'backward'},
                                  {'count': 12, 'label': '1Y', 'step': 'month', 'stepmode': 'backward'},
                                  {'step': 'all'}])}}
                      )
)

# Put everything together
VBox((HBox(),f))

ValueError: 
    Invalid value of type 'builtins.str' received for the 'step' property of layout.xaxis.rangeselector.button
        Received value: 'week'

    The 'step' property is an enumeration that may be specified as:
      - One of the following enumeration values:
            ['month', 'year', 'day', 'hour', 'minute', 'second',
            'all']